In [3]:
from tensorflow import keras
keras.__version__

'2.2.4-tf'

# Un premier regard sur un réseau de neurones

- Premier exemple concret de réseau de neurones, qui utilise le package Python Keras pour apprendre à classer chiffres écrits à la main. 


- Le problème que nous essayons de résoudre ici est de classer les images en niveaux de gris de chiffres manuscrits (28 pixels sur 28 pixels) dans leur 10 catégories (0 à 9). 


- Le jeu de données que nous allons utiliser est le MNIST, un jeu de données classique de la communauté d’apprentissage automatique, qui a été presque aussi longtemps que le champ lui-même et a été très intensément étudié. C'est un ensemble de 60 000 images d'apprentissage, plus 10 000 de tests

In [5]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Etudions les données d'apprentissage :

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(train_labels)

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Notre flux de travail sera le suivant: d’abord, nous présenterons à notre réseau de neurones les données d’entraînement, `train_images` et` train_labels`.


Le réseau apprendra ensuite à associer des images et des étiquettes. Enfin, nous demanderons au réseau de produire des prédictions pour `test_images`, et nous vérifierons si ces prédictions correspondent aux étiquettes de `test_labels`.

In [6]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

Ici, notre réseau consiste en une séquence de deux couches `Dense`, qui sont des couches densément connectées.


La deuxième (et dernière) couche est une couche "softmax", ce qui signifie qu’elle renverra un tableau de 10 scores de probabilité


Nous devons choisir trois éléments supplémentaires dans le cadre de l’étape "compilation":

- Une fonction de perte: voici comment le réseau sera capable de mesurer la qualité de son travail sur ses données de formation, et donc comment il le sera capable de se diriger dans la bonne direction.
-  Un optimiseur: c'est le mécanisme par lequel le réseau se mettra à jour en fonction des données qu'il voit et de sa fonction de perte.
- Mesures : pendant la formation et les tests. Ici, nous ne nous intéresserons qu'à la précision (la fraction des images qui ont été correctement classifié).

In [7]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

Avant l'apprentissage, nous allons prétraiter nos données en les transformant dans la forme attendue par le réseau et en les redimensionnant de manière à ce que toutes les valeurs soient en mémoire.


Nos images étaient stockées dans un tableau de formes `(60000, 28, 28)` de type `uint8` avec des valeurs dans l'intervalle `[0, 255]`. Nous le transformons en un `tableau float32` de forme` (60000, 28 * 28) `avec des valeurs comprises entre 0 et 1.

In [8]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255


Nous devons également coder les étiquettes de manière catégorielle : 

In [9]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Nous sommes maintenant prêts à apprendre notre réseau via un appel à la méthode `fit` du réseau :

In [10]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 6s 106us/step - loss: 0.2602 - acc: 0.9247
Epoch 2/5
60000/60000 [==============================] - 6s 97us/step - loss: 0.1045 - acc: 0.9686
Epoch 3/5
60000/60000 [==============================] - 6s 97us/step - loss: 0.0685 - acc: 0.9803
Epoch 4/5
60000/60000 [==============================] - 6s 96us/step - loss: 0.0497 - acc: 0.9851
Epoch 5/5
60000/60000 [==============================] - 6s 95us/step - loss: 0.0369 - acc: 0.9890


Deux quantités sont affichées pendant l’apprentissage : la "perte" du réseau sur les données d’apprentissage et la précision du réseau sur les données d'entraînement.

Nous atteignons rapidement une précision de 0,989 (soit 98,9%) sur les données d’apprentissage. Vérifions maintenant que notre modèle fonctionne bien sur le jeu de tests:

In [11]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 75us/step


In [12]:
print('test_acc:', test_acc)

test_acc: 0.9813
